# Udacity Data Wrangling Project 

## Objective

In this project we are going to extract data from different sources related with the Twitter account `@dog_rates`.

Basicly we will work with four data sources:

* **'twitter-archive-enhanced.csv':**  A csv file with 2356 tweets of this account. Each one with a picture of a dog. They use to mark this dogs, usually with marks greater than 10 over 10: 11/10, 13/10, etc. This file is provided by Udacity for making the project.

* **'image-predictions.tsv':** A tsv file with the results obtained of applying a predictive method over the pictures of the tweets. This file was obtained in a project in another nanodegreee and it is provided by Udacity also. Every image in the WeRateDogs Twitter archive was run through a neural network that can classify breeds of dogs. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

* **Additional information obtained with tweetpy:** Once obtained the Twitter credentials, we are going to use the tweetpy API to get more additional data. To do this we will connect to the Twitter platform and using tweetpy we will download the tweet status for each tweet in `twitter-archive-enhanced.csv`. Then we will save these results in a file callde `twitter_archive.json` using the json library. Finally, we will read this file and we will extract some more data to another dataframe using json again.

* **Information abour the replies of each tweet:** Finally we would like to extract the data corresponding to the replies of each tweet. We have tried some different methods:

    * In some places it is recommended to use tweepy to make a query of all the tweets referenced to @rate_dogs, and search which of them are a reply to the status of the tweet. Translated to code, something like this:
    
            consumer_key = 'XXXXXX'
            consumer_secret = 'XXXXXX'
            access_token = 'XXXXXX'
            access_secret = 'XXXXXX'

            auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_secret)

            twapi = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
            replies=[]

            for tweet in tweepy.Cursor(twapi.search,q='to:'+name, since_id=892420643555336193, result_type='recent',timeout=999999).items(1000)
                if hasattr(tweet, 'in_reply_to_status_id_str'):
                    if (tweet.in_reply_to_status_id_str==tweet_id):
                        replies.append(tweet)
            
      but it has a lot of limitations and I didn't like it too much.

    * In other places it is recommended to use the urllib3 library to request pages. Then, you can use BeautifulSoup to interpret the result and scrapp the information that you need:
    
             http = urllib3.PoolManager()
             url = "https://twitter.com/dog_rates/status/892420643555336193"
             r = http.request('GET', url)
             soup = BeautifulSoup(r.data)
             tweets = soup.find_all('li','js-stream-item')
             for tweet in tweets:

             full_name = tweet.find("span", "FullNameGroup").find("strong", "fullname").contents[0]  
        
      But, in this case, you need to make scroll down on the page to see all the replies. Even so, when there are too many replies, the page cut the list and ask you in a link if you want to see more. You had to do this as many times as you need until you reach the end of the list. Apart from that, sometimes there are replies to the replies, and the page has another link to select to see them. I mean that with a single request you can't see all the replies if these are a lot.
      Maybe yo can do that using additional requests with POST or some other commands and sending the correct instruction to click in all the necessary links. But I felt like it was too much complicated.
       
    * Finally, I tried another method to do scrapping. I used the `selenium` library. It permits you to use a local browser to open the pages. You can navigate using the program on these pages and select and click any element of the page. Once you have deployed completely the page you can get it to a beautifulsoup object and interpret it. And using a local browser can be viewed as a disadvantage, but I felt more comfortable with this method and it is what I have used.




### Imports

In [1]:
#basic data libraries
import pandas as pd
import numpy as np
#to interact with the local system
import os
import sys
#to work with regular expressions
import re
#imports to user timers and make conversions of time formats
from timeit import default_timer as timer
from datetime import datetime
import time
#to make logs and track those processes that take a long time
import logging
#to get and interpret information of the web
import requests
import tweepy
from tweepy import OAuthHandler
import json
from bs4 import BeautifulSoup
import urllib.parse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException



### Gathering The Data

* The first step will be to define a function to connect to Twitter using the API `tweepy`. We will use this function in other cells below. 

In [2]:
def connect_twitter():
    '''
    It connects to Twitter API.
    
    Returns:
        twapi: tweepy.api object to interact with the page.
    '''
    
    #It reads the keys to connect to Twitter API from a local file.
    #These keys are hidden to comply with Twitter's API terms and conditions
    with open('API keys.txt', mode = 'r') as file:
        keys = file.readlines()
        
    keys = [x.strip() for x in keys] 
    
    consumer_key = keys[0].split(":")[1]
    consumer_secret = keys[1].split(":")[1]
    access_token = keys[2].split(":")[1]
    access_secret = keys[3].split(":")[1]
    
    #It authenticates in tweepy with the previous credentials.
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twapi = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    return twapi


* Create the folder where to save the necesary files.

In [3]:
#It creates a folder called resources if it does not exists
folder_name = 'resources'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


* Load data from `twitter-archive-enhanced.csv` file supplied by Udacity. This dataframe has the following columns:

    - **tweet_id:** The integer representation of the unique identifier for this Tweet. 
    - **in_reply_to_status_id:** If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s ID.
    - **in_reply_to_user_id:**  If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet.
    - **timestamp:** date and time of the tweet.
    - **source:** Utility used to post the Tweet, as an HTML-formatted string. Tweets from the Twitter website have a source value of web.
    - **text:** The actual UTF-8 text of the status update. 
    - **retweeted_status_id:** If the represented Tweet is a retweet, this field will contain the integer representation of the original Tweet’s ID. If it is a retweet of a retweet it containg the original message id.
    - **retweeted_status_user_id:**  If the represented Tweet is a retweet, this field will contain the integer representation of the original Tweet’s author ID. This will not necessarily always be the user directly mentioned in the Tweet.
    - **retweeted_status_timestamp:** If the represented Tweet is a retweet, the timestampo of the original tweet.
    - **expanded_urls:** url of the tweet.
    - **rating_numerator:** numerator of the rating assigned according to the text of the tweet.
    - **rating_denominator:** denominator of the rating assigned according to the text of the tweet.
    - **name:** name of the dog according to the text of the tweet.
    - **doggo:** type of the dog acording to the text and to the clasification used in the page.
    - **floofer:** type of the dog acording to the text and to the clasification used in the page.
    - **pupper:** type of the dog acording to the text and to the clasification used in the page.
    - **puppo:** type of the dog acording to the text and to the clasification used in the page. 
    
    
![alt text](dogtionary-combined.png)


In [4]:
#Load the file twitter-archive-enhanced.csv into a dataframe
df_twitter_archive_enhanced = pd.read_csv(os.path.join(folder_name, 'twitter-archive-enhanced.csv'))

* Load data from the `image-predictions.tsv` file. This file was provided by Udacity in a especified url.

In [5]:
#We donwload the image-predictions.tsv file from the expecified url.
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
        file.write(response.content)

* We crate a new dataframe called `df_image_predictions`. This dataframe has the following columns:

    - **tweet_id:** tweet_id is the last part of the tweet URL after "status/": `https://twitter.com/dog_rates/status/889531135344209921`
    
    - **jpg_url:** url of the image of the tweet. It can be downloaded.
    - **img_num:** the image with the most confident prediction.
    - **p1:** is the algorithm's #1 prediction for the image in the tweet.
    - **p1_conf:** is how confident the algorithm is in its #1 prediction.
    - **p1_dog:** is whether or not the #1 prediction is a breed of dog.
    - **p2:** is the algorithm's second most likely prediction.
    - **p2_conf:** is how confident the algorithm is in its #2 prediction.
    - **p2_dog:** is whether or not the #2 prediction is a breed of dog.
    - **p3:** is the algorithm's third most likely prediction.
    - **p3_conf:** is how confident the algorithm is in its #3 prediction.
    - **p3_dog:** is whether or not the #3 prediction is a breed of dog.

In [6]:
#Load the file image-predictions.tsv into the dataframe df_image_predictions
df_image_predictions = pd.read_csv(os.path.join(folder_name, 'image-predictions.tsv'), sep='\t')

* Load additional data from the pages of the tweets with the API tweepy. At the same time, we can see which pages still exist and which are not available in this moment. First, we download the content of the page of each tweet and we store it in a file calle `twitter_archive.json`, in json format.

In [7]:
=================================================
REMOVE THIS TO EXECUTE IT. IT CAN TAKES MORE THAN AN HOUR.
=================================================
#we connect to the Twitter API using tweepy 
twapi = connect_twitter()

start = timer()
#It configures the file 'tweepy_api.log' as a log to track the evolution
logging.basicConfig(filename='tweepy_api.log',level=logging.DEBUG)

#It initializes a list of Id's with all the tweets.
list_ids = df_twitter_archive_enhanced.tweet_id

total_count = 0
error_count = 0

#It initializes the log file
open('tweepy_api.log', 'w').close()

#It initializes the files used to save the results.
#We have created two files: one estructured in lines and other indented for a more
#friendly check.
open(os.path.join(folder_name, 'twitter_archive_indent.json'), 'w').close()
open(os.path.join(folder_name, 'twitter_archive.json'), 'w').close()

for tweet_id in list_ids:
    total_count += 1
    logging.debug('%s: Trying tweet for ID %s', total_count, tweet_id)
    try:
        #download the content of a tweet for a tweet_id given
        tweet = twapi.get_status(tweet_id, tweet_mode='extended')
        #store the content of the tweet using json in the file tweet_json_indent.txt, indent=2 spaces.
        with open(os.path.join(folder_name, 'twitter_archive_indent.json'), 'a', encoding='utf8', newline='\n') as out_file:
            json.dump(tweet._json, out_file, indent=2, ensure_ascii=False)
            out_file.write('\n')
        #store the content of the tweet using json in the file tweet_json.txt, in a sigle line.
        with open(os.path.join(folder_name, 'twitter_archive.json'), 'a', encoding='utf8', newline='\n') as out_file:
            json.dump(tweet._json, out_file, ensure_ascii=False)
            out_file.write('\n')
            
        #separate each tweet in the log file.    
        logging.debug('============================================================================')
        logging.debug('============================================================================')
    except tweepy.TweepError as te:
        #if we cannot download the tweet, we reflect this in the log and we increment the error count.
        logging.warning('%s: FAILED to get tweet ID %s: %s', total_count, tweet_id, str(te))
        error_count += 1
                    
    end = timer()
    #separate each tweet in the log file. 
    logging.debug('TOTAL: %s: TIME %s%s', total_count, end-start,'===========================================')
    logging.debug('TOTAL: %s. ERRORS: %s%s', total_count, error_count,'===========================================')
    logging.debug('============================================================================')
    logging.debug('============================================================================')
        


SyntaxError: invalid syntax (<ipython-input-7-c87b82675463>, line 1)

* Total count of tweets in this new file and number of errors. The errors are pages that existed when Udacity extracted the file `twitter-archive-enhanced.csv`, but now are unavailable.

In [ ]:
total_count, error_count

* Use the data stored in the file `twitter_archive.json` in the previous step to create a new dataframe called `df_tweepy_extractions`. This dataframe will have the following columns:

    - **tweet_id:** The integer representation of the unique identifier for this Tweet.
    - **entities_name:** Users who are labelled under the picture of the tweet.
    - **entities_screen_name:** Screen name of the users who are labelled under the picture of the tweet.
    - **entities_type:** The type of the entity. In this case is always 'user'
    - **entities_user_id:** ID of the users who are labelled under the picture of the tweet.
    - **favorite_count:**  Indicates approximately how many times this Tweet has been liked by Twitter users. 
    - **favorites_count_retweet:** This field only surfaces when the Tweet is a retweet. Indicates approximately how many times the original Tweet has been liked by Twitter users. 
    - **mentions_name:** Display name of the referenced user in the text of the tweet.
    - **mentions_screen_name:** Screen name of the referenced user in the text of the tweet.
    - **mentions_user_id:** ID of the user mentioned in the text of the tweet.
    - **quoted_status_id:** This field only surfaces when the Tweet is a quote Tweet. This field contains the integer value Tweet ID of the quoted Tweet. 
    - **quoted_user_id:** ID of the user quoted.
    - **quoted_status_id_rwetweet:** This field only surfaces when the Tweet is a retweet and the original Tweet is a is a quote Tweet. This field contains the integer value Tweet ID of the quoted Tweet.
    - **retweet_count:** Number of times this Tweet has been retweeted.
    - **retweet_count_retweet:** This field only surfaces when the Tweet is a retweet. Indicates approximately how many times the original Tweet has been retweeted.
    

In [8]:
#we first read the file and load the lines in a list called content
with open(os.path.join(folder_name, 'twitter_archive.json'), 'r', encoding='utf8') as input_file:
    content = input_file.readlines()
content = [x.strip() for x in content] 

#initialize the result dataframe df_tweepy_extractions
df_tweepy_extractions = pd.DataFrame()
tweet_status = {}

#read recursively each line in the list content
for line in content:
    #initialize the outcomes
    entities_name = ''
    entities_screen_name  = ''
    entities_type = ''
    entities_user_id = ''
    mentions_user_id = ''
    mentions_name = ''
    mentions_screen_name = ''
    favorites_count_retweet = 0
    retweet_count_retweet = 0
    quoted_status_id_rwetweet = ''
    quoted_status_id_str = ''
    quoted_user_id_str = ''
    quoted_user_id_retweet = ''
    
    #read each string with json.loads to interpretarte it
    tweet_status = json.loads(line)
    try:
        #if the object media exists innside entities.
        if 'media' in tweet_status['entities']:
            #In this case we are going to see the user or users tagged below the picture
            if 'all' in tweet_status['entities']['media'][0]['features']:
                #a list with the names of the users tagged
                entities_name = [name['name'] for name in tweet_status['entities']['media'][0]['features']['all']['tags']]
                #the screen names eje:@bla_bla_bla
                entities_screen_name = [screen_name['screen_name'] for screen_name in tweet_status['entities']['media'][0]['features']['all']['tags']]
                #types: user
                entities_type = [type_['type'] for type_ in tweet_status['entities']['media'][0]['features']['all']['tags']]
                #a list with the users id
                entities_user_id = [user_id['user_id'] for user_id in tweet_status['entities']['media'][0]['features']['all']['tags']]
        #if the object user_mentions exists innside entities. We can search the users named inside te text part.
        if 'user_mentions' in tweet_status['entities']:
            if len(tweet_status['entities']['user_mentions']) > 0:
                #the id of the user mentioned
                mentions_user_id = [user_id['id_str'] for user_id in tweet_status['entities']['user_mentions']]
                #the name of the user mentioned
                mentions_name = [name['name'] for name in tweet_status['entities']['user_mentions']]
                #the screen name of the user mentioned
                mentions_screen_name = [screen_name['name'] for screen_name in tweet_status['entities']['user_mentions']]
        #if the tweet is a retweet
        if 'retweeted_status' in tweet_status:
            if 'favorite_count' in tweet_status['retweeted_status']:
                #number of favorites in the original tweet.
                favorites_count_retweet = tweet_status['retweeted_status']['favorite_count']
                #number of retweets in the original tweet.
                retweet_count_retweet = tweet_status['retweeted_status']['retweet_count']
            if 'quoted_status_id_str' in tweet_status['retweeted_status']:
                #if the tweet is a retweet of a previously quoted tweet. The tweet id of the original quoted tweet.
                quoted_status_id_rwetweet = tweet_status['retweeted_status']['quoted_status_id_str']
                if 'quoted_status' in tweet_status['retweeted_status']:
                    #id of the user quoted.
                    quoted_user_id_retweet = tweet_status['retweeted_status']['quoted_status']['user']['id_str']
    
        #if the tweet is a quoted ot other tweet.
        if 'quoted_status_id_str' in tweet_status:
            #the id of the quoted tweet.
            quoted_status_id_str = tweet_status['quoted_status_id_str']
            if 'quoted_status' in tweet_status:
                #id of the user quoted.
                quoted_user_id_str = tweet_status['quoted_status']['user']['id_str']
    
    except Exception as e:
        #register in the log any exception that it can occurs.
        print(tweet_status['id_str'])
        logging.warning(e)
    #save the results in the dataframe df_tweepy_extractions
    df_tweepy_extractions = df_tweepy_extractions.append({'tweet_id': tweet_status['id_str'],
                                                          'retweet_count': tweet_status['retweet_count'],
                                                          'favorite_count': tweet_status['favorite_count'],
                                                          'favorites_count_retweet': favorites_count_retweet,
                                                          'retweet_count_retweet': retweet_count_retweet,
                                                          'entities_name': entities_name,
                                                          'entities_screen_name': entities_screen_name,
                                                          'entities_type': entities_type,
                                                          'entities_user_id': entities_user_id,
                                                          'mentions_user_id': mentions_user_id,
                                                          'mentions_name': mentions_name,
                                                          'mentions_screen_name': mentions_screen_name,
                                                          'quoted_status_id': quoted_status_id_str,
                                                          'quoted_user_id': quoted_user_id_str,
                                                          'quoted_status_id_rwetweet': quoted_status_id_rwetweet,
                                                          'quoted_user_id_retweet': quoted_user_id_retweet
                                                          },ignore_index=True)
    #It sorts the columns of the dataframe. 
    df_tweepy_extractions = df_tweepy_extractions[['tweet_id', 'retweet_count', 'favorite_count', 'favorites_count_retweet',
                                                 'retweet_count_retweet', 'entities_name', 'entities_screen_name', 
                                                 'entities_type', 'entities_user_id', 'mentions_user_id', 'mentions_name',
                                                 'mentions_screen_name', 'quoted_status_id', 'quoted_user_id', 
                                                 'quoted_status_id_rwetweet', 'quoted_user_id_retweet']]


In [9]:
df_tweepy_extractions.query('quoted_user_id_retweet != ""')

,tweet_id,retweet_count,favorite_count,favorites_count_retweet,retweet_count_retweet,entities_name,entities_screen_name,entities_type,entities_user_id,mentions_user_id,mentions_name,mentions_screen_name,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet
31,886054160059072513,101.0,0.0,1482.0,101.0,,,,,[19607400],[Oakland A's],[Oakland A's],886053434075471873,,886053434075471873,4196983835


#### Scrapping Replies Using Selenium

* Finally we are going to sacrap more information about each tweet using the library selenium. We are interested into obtain information about all the replies for each tweet. We will get a final dataframe called `df_scrapped_replies` with the following columns:

    - **conversation:** Id of the replied tweet.
    - **favs:** Number of favorites for this replying tweet.
    - **full_name:** name of the user who has replied.
    - **image:** If there is an image in the reply, it especifies the url.
    - **language:** When present, indicates a BCP 47 language identifier corresponding to the machine-detected language of the Tweet text.
    - **references:** Other users ID that are referenced in the text of the reply, if they exist.
    - **replies:** Number of replies to this reply.
    - **reply_id:** tweet ID for this reply.
    - **retweets:** Number of retweets of this reply.
    - **text:** Text include in the reply.
    - **timestamp:** Date_time of the reply.
    - **user_id:** Id of the user who has replied.
    - **user_name:** Name of the user who has replied (@XXXXX).


* This fuction download a status page with all its replies into a driver object. We have user the Firefox driver.

In [10]:
def download_page(driver, user_name, conversation_id):

    '''
    This fuction download a status page with all its replies into a driver object. We have user the Firefox driver.
    
    Args:
        driver: selenium.webdriver object. Used to get the page, to move on it and make actions.
        (str) user_name: user name of the twitter profile. In this project: dog_rates.
        (str) conversation_id: status or tweet id for which we want to extract the data.
    
    Return:
    
        driver: selenium.webdriver object. The same object, but with all the replies to the tweet opened
    
    '''
    
    #Initialize a new file for the log.
    logging.basicConfig(filename='scrapping_replies.log',level=logging.DEBUG)
    #url of the page that we want to download
    url = "https://twitter.com/" + user_name + "/status/" + conversation_id
    #time to wait after each scroll
    SCROLL_PAUSE_TIME = 2
    # tells WebDriver to poll the DOM for a certain amount of time when trying 
    #to find any element (or elements) not immediately available.
    driver.implicitly_wait(10)
    
    driver.get(url)
    #we use the length of the page to know if we have downloaded the complete page.
    last_length = len(driver.page_source)
    
    count = 0
    while True:
        #scroolls down to the end of the page
        driver.find_element_by_tag_name("body").send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        #the page is actualized with more replies. We get the new length
        new_length = len(driver.page_source)
        count += 1
        if count == 6:
            count = 0
        #Even when we have reached the end we wait 4*two weconds, just in case the page is not complete downloaded
        if (new_length == last_length) & (count > 4):
            count = 0
            #When the page is completely downloaded and there are not more replies to show, 
            #we search for the le link 'show nore replies' and we click it.
            try:             
                button_more = driver.find_element_by_css_selector('.ThreadedConversation-showMoreThreadsButton.u-textUserColor')
                button_more.click()
            except (NoSuchElementException, AttributeError) as e:
                logging.warning(e)
                break
        #se set the old length equal to the new length to start again the scroll down process.
        last_length = new_length
    #When there are not more replies neither any 'Show nore replies' link.
    #We click in all the intermediate links 'x replies more'. Sometimes people replies to the replies and this other replies
    #are not always showed at first.
    try:
        links_replies = driver.find_elements_by_css_selector('.ThreadedConversation-moreRepliesLink')
        for link in links_replies:
            link.click()
    except (NoSuchElementException, AttributeError) as e:
        logging.warning(e)
    #return the driver object with the complete page.
    return driver

* This function searchs for information inside the page stored in driver and it saves the content in the dataframe `df_scrapping_replies`.

In [11]:
def analize_page(driver, conversation_id):
    
    '''
    It searchs for information inside the page stored in driver and it saves the content in the dataframe df_scrapping_replies.
    
    Args:
        driver: selenium.webdriver object. Used to get the page, to move on it and make actions.
        (str) conversation_id: status or tweet id for which we want to extract the data.
    
    Return:
    
        df_scrapping_replies: pandas.dataframe object with all the data gathered.
    '''
    
    #it configures the file for the log.
    logging.basicConfig(filename='scrapping_replies.log',level=logging.DEBUG)
    #it initializes the dataframe df_scrapping_replies
    df_scrapping_replies = pd.DataFrame()
    #call BeautifulSoup with the driver page to be decoded
    soup = BeautifulSoup(driver.page_source, "html.parser")
    #load the content in a list of replies
    tweets = soup.find_all('li','js-stream-item')

    for tweet in tweets:
        #analyze each reply one by one
        try:
            #the status id of the reply
            reply_id = tweet.get('data-item-id')
            full_name = ""
            #we get only the text part of the name. There is other parts like emojis.
            full_names = tweet.find("span", "FullNameGroup").find("strong", "fullname").contents
            for name in full_names:
                if isinstance(name,  str):
                    full_name = full_name + name
                    
            #the name with @
            user_name = tweet.find("span", "username").find("b").contents[0].strip()
            #the id of the user.
            user_id = tweet.find("div",class_=re.compile("^tweet js-stream-tweet")).get('data-user-id')
            logging.debug(user_id)
            #The number of replies to this reply, number ot retweets and faver of this reply.
            replies = tweet.find("span", id=re.compile("^profile-tweet-action-reply-count")).contents[0]
            retweets = tweet.find("span", id=re.compile("^profile-tweet-action-retweet-count")).contents[0]
            favs = tweet.find("span", id=re.compile("^profile-tweet-action-favorite-count")).contents[0]
            #The language of the message if it is configured.
            language = tweet.find("p", "TweetTextSize js-tweet-text tweet-text").get('lang')
            #In the text part we only get the str part. We discard emojis and other things.
            texts = tweet.find("p", "TweetTextSize js-tweet-text tweet-text").contents
            text = ""
            ref = ""
            image = ""
            for subtext in texts:
                if isinstance(subtext,  str):
                    text = text + subtext
            ref_aux = tweet.find_all("a", "pretty-link js-user-profile-link")
            #the other users id that are referred in the text of the reply
            for subref in ref_aux:
                subrefs = subref.get('data-user-id')
                ref.append(subrefs)
            #We try to find if there is some image attached. If it is so, we save the url of the picture.
            try:
                image = tweet.find("div", "AdaptiveMedia-photoContainer js-adaptive-photo").get('data-image-url') 
            except AttributeError as e:
                logging.warning(e)
            #we also get the date_time of the reply
            timestamp = tweet.find("small", "time").find("span", "_timestamp js-short-timestamp").get('data-time')
            timestamp = str(datetime.fromtimestamp(int(timestamp)))
            
            #se save all in the dataframe df_scrapping_replies 
            df_scrapping_replies = df_scrapping_replies.append({'timestamp': timestamp,
                                                                'conversation': conversation_id,
                                                                'reply_id': reply_id,
                                                                'full_name': full_name,
                                                                'user_name': '@' + user_name,
                                                                'user_id': user_id,
                                                                'image': image,
                                                                'replies': int(replies.split()[0].replace('.','')),
                                                                'retweets': int(retweets.split()[0].replace('.','')),
                                                                'favs': int(favs.split()[0].replace('.','')),
                                                                'text': text,
                                                                'language': language,
                                                                'references': ref
                                                                },ignore_index=True)

        except AttributeError as e:
            logging.warning(e)
    
    #It reurns df_scrapping_replies 
    return df_scrapping_replies
    

* We call the neccessary functions to make the scrap. As it takes a long time, each time that we search the replies of a tweet we save them in the csv file. Instead of saving the complete file at the end. If there were any problem we would have save the results until this moment.

In [12]:
=================================================
REMOVE THIS TO EXECUTE IT. IT CAN TAKES SEVERAL HOURS TO FINISH
=================================================
start = timer()
#initialize the log file
open('scrapping_replies.log', 'w').close()
logging.basicConfig(filename='scrapping_replies.log',level=logging.DEBUG)

logging.debug("********************** CONNECTING DRIVER TO PAGE **********************")
#initialize the driver.
twapi = connect_twitter()
driver = webdriver.Firefox()
logging.debug("********************** CONNECTION DONE!!!! **********************")
user_name = 'dog_rates'
i = 1
try:
    #search for each tweet id in df_tweepy_extractions (the tweets that we know that are available in this moment)
    for tweet_id in df_tweepy_extractions['tweet_id']:
        #call the download_page function to download the complete page
        logging.debug("********************** START DOWNLOAD: " + tweet_id + "(" + str(i) + ")" + ' **********************')
        driver = download_page(driver, user_name, tweet_id)
        logging.debug("********************** END DOWNLOAD: " + tweet_id + "(" + str(i) + ")" + ' **********************')
        #call the analize_page function to extract a dataframe with the results
        logging.debug("********************** START ANALYSIS: " + tweet_id + "(" + str(i) + ")" + ' **********************')
        df_scrapping_replies = analize_page(driver, tweet_id)
        #The first time that we save the resutls in scrapped_replies.csv we open the file in write mode and we write a header.
        #The next times we open the file in append mode and we don't write the header.
        if i == 1:
            df_scrapping_replies.to_csv(os.path.join(folder_name, 'scrapped_replies.csv'), mode='w', encoding='utf-8', index=False)
        else:
            df_scrapping_replies.to_csv(os.path.join(folder_name, 'scrapped_replies.csv'), mode='a', encoding='utf-8', index=False, header = False)
        end = timer()
        #each time we save a result. we register in the log file the number of tweet_id analyzed and the time consumed.
        logging.debug("********************** END ANALYSIS: " + tweet_id + "(" + str(i) + ")" + str(start-end) + ' **********************')
        i += 1
except Exception as e:
    logging.warning(e)
finally:
    #close and disconnect the driver.
    driver.quit()


SyntaxError: invalid syntax (<ipython-input-12-380484fc4cee>, line 1)

* We load the data saved in `scrapped_replies.csv` in the steps before to the dataframe `df_scrapped_replies`. We can see that there are 23 tweets with no replies. We have 2310 rows in df_scrapped_replies and 2333 in df_tweepy_extractions.

In [13]:
df_scrapped_replies = pd.read_csv(os.path.join(folder_name, 'scrapped_replies.csv'))
df_scrapped_replies.conversation.nunique()

2310

In [14]:
df_tweepy_extractions.shape

(2333, 16)

These are the list of tweets without any replies.

In [15]:
df_scrapped_replies.conversation = df_scrapped_replies.conversation.astype(str)
df_tweepy_extractions[~df_tweepy_extractions.tweet_id.isin(df_scrapped_replies.conversation)].tweet_id

1941    673350198937153538
2054    670833812859932673
2063    670803562457407488
2184    668627278264475648
2190    668567822092664832
2193    668537837512433665
2198    668480044826800133
2201    668291999406125056
2241    667538891197542400
2245    667517642048163840
2257    667393430834667520
2264    667177989038297088
2286    666804364988780544
2289    666776908487630848
2292    666691418707132416
2302    666418789513326592
2304    666407126856765440
2316    666102155909144576
2318    666094000022159362
2320    666073100786774016
2325    666055525042405380
2327    666050758794694657
2331    666029285002620928
Name: tweet_id, dtype: object

## Assesing The Data

* `df_twitter_archive_enhanced`

In [16]:
df_twitter_archive_enhanced.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
585,800018252395122689,NaN,NaN,2016-11-19 16:49:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo doin a struggle. 11/10 much det...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/800018252...,11,10,None,doggo,None,None,None
595,798701998996647937,NaN,NaN,2016-11-16 01:39:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: We normally don't rate marshmal...,7.186315e+17,4.196984e+09,2016-04-09 02:47:55 +0000,https://twitter.com/dog_rates/status/718631497...,10,10,None,None,None,None,None
923,755955933503782912,NaN,NaN,2016-07-21 02:41:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a frustrated pupper attempting to escap...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/755955933...,12,10,None,None,None,pupper,None
378,827933404142436356,NaN,NaN,2017-02-04 17:34:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Kona. Yesterday she stopped by the dep...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/827933404...,12,10,Kona,None,None,pupper,None
418,822462944365645825,NaN,NaN,2017-01-20 15:17:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gabe. He was the unequivocal embodimen...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/822462944...,14,10,Gabe,None,None,pupper,None


In [17]:
df_twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [18]:
#Text for the retweeted tweets.
df_twitter_archive_enhanced.query('retweeted_status_id.notnull() == True').text

19      RT @dog_rates: This is Canela. She attempted s...
32      RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...
36      RT @dog_rates: This is Lilly. She just paralle...
68      RT @dog_rates: This is Emmy. She was adopted t...
73      RT @dog_rates: Meet Shadow. In an attempt to r...
                              ...                        
1023    RT @dog_rates: This is Shaggy. He knows exactl...
1043    RT @dog_rates: Extremely intelligent dog here....
1242    RT @twitter: @dog_rates Awesome Tweet! 12/10. ...
2259    RT @dogratingrating: Exceptional talent. Origi...
2260    RT @dogratingrating: Unoriginal idea. Blatant ...
Name: text, Length: 181, dtype: object

In [19]:
#unique numerators and denominators.
df_twitter_archive_enhanced.rating_numerator.unique(), df_twitter_archive_enhanced.rating_denominator.unique()

(array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
         182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
           3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
         143,  121,   20,   26,    2,  144,   88], dtype=int64),
 array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
        130, 110,  16, 120,   2], dtype=int64))

In [20]:
#numerator when the denominator = 10
df_twitter_archive_enhanced.query('rating_denominator == 10').rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,    0,   75,    7,    9,    8,    1,   27,    3,    4, 1776,
         26,    2], dtype=int64)

In [21]:
#names with length smaller than 4 chars
df_twitter_archive_enhanced.query('name.str.len() < 4').name.unique()

array(['Jax', 'Ted', 'Jim', 'Gus', 'Rey', 'a', 'Aja', 'Jed', 'Leo', 'Ken',
       'Max', 'Ava', 'Eli', 'Ash', 'not', 'Mia', 'one', 'Ike', 'Mo', 'Bo',
       'Tom', 'Alf', 'Sky', 'Tyr', 'Moe', 'Sam', 'Ito', 'Doc', 'mad',
       'Jay', 'Mya', 'an', 'O', 'Al', 'Lou', 'my', 'Eve', 'Dex', 'Ace',
       'Zoe', 'Blu', 'his', 'all', 'Sid', 'old', 'Ole', 'Bob', 'the',
       'Obi', 'by', 'Evy', 'Tug', 'Jeb', 'Dot', 'Mac', 'Ed', 'Taz', 'Cal',
       'JD', 'Pip', 'Amy', 'Gin', 'Edd', 'Ben', 'Dug', 'Jo', 'Ron', 'Stu'],
      dtype=object)

In [22]:
#text for dogs with name 'O'
df_twitter_archive_enhanced.query('name == "O"').text

775    This is O'Malley. That is how he sleeps. Doesn...
Name: text, dtype: object

In [23]:
#text for dogs with name 'by'
df_twitter_archive_enhanced.query('name == "by"').text

1724    This is by far the most coordinated series of ...
Name: text, dtype: object

In [24]:
#sum of number of dogs for each clasification
t = (df_twitter_archive_enhanced.query('doggo != "None"').tweet_id.count(),
df_twitter_archive_enhanced.query('floofer != "None"').floofer.count(),
df_twitter_archive_enhanced.query('pupper != "None"').pupper.count(),
df_twitter_archive_enhanced.query('puppo != "None"').puppo.count())
sum(t)

394

In [25]:
#number of dogs with at least one clasification.
df_twitter_archive_enhanced[df_twitter_archive_enhanced['doggo'].str.contains("doggo") | 
                            df_twitter_archive_enhanced['floofer'].str.contains("floofer") |
                            df_twitter_archive_enhanced['pupper'].str.contains("pupper") |
                            df_twitter_archive_enhanced['puppo'].str.contains("puppo")].shape

(380, 17)

In [26]:
#tweets with two clasifications
df_twitter_archive_enhanced[
    df_twitter_archive_enhanced[["doggo","floofer","pupper","puppo"]].
                            isin(["doggo","floofer","pupper","puppo"]).sum(axis=1)> 1][['tweet_id',"doggo","floofer","pupper","puppo"]]

,tweet_id,doggo,floofer,pupper,puppo
191,855851453814013952,doggo,None,None,puppo
200,854010172552949760,doggo,floofer,None,None
460,817777686764523521,doggo,None,pupper,None
531,808106460588765185,doggo,None,pupper,None
565,802265048156610565,doggo,None,pupper,None
575,801115127852503040,doggo,None,pupper,None
705,785639753186217984,doggo,None,pupper,None
733,781308096455073793,doggo,None,pupper,None
778,775898661951791106,doggo,None,pupper,None
822,770093767776997377,doggo,None,pupper,None


* `df_image_predictions`

In [27]:
df_image_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1698,816816676327063552,https://pbs.twimg.com/media/C1XqbhXXUAElpfI.jpg,1,malamute,0.668164,True,Pembroke,0.105033,True,Siberian_husky,0.077875,True
832,693647888581312512,https://pbs.twimg.com/media/CaBVE80WAAA8sGk.jpg,1,washbasin,0.272451,False,doormat,0.165871,False,bathtub,0.066368,False
663,682662431982772225,https://pbs.twimg.com/media/CXlN1-EWMAQdwXK.jpg,1,beagle,0.413824,True,Cardigan,0.263553,True,basset,0.167618,True
1762,826204788643753985,https://pbs.twimg.com/media/C3dEza1WcAAhlNU.jpg,2,Labrador_retriever,0.782058,True,golden_retriever,0.156581,True,soft-coated_wheaten_terrier,0.007275,True
1738,822244816520155136,https://pbs.twimg.com/media/C2kzTGxWEAEOpPL.jpg,1,Samoyed,0.585441,True,Pomeranian,0.193654,True,Arctic_fox,0.071648,False


In [28]:
df_image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [29]:
#tweets that have not been predicted as a dog in any of the three predictions.
df_image_predictions.query('p1_dog == False & p2_dog == False & p3_dog == False')[['img_num',
                                                                                   'tweet_id',
                                                                                   'p1','p1_dog',
                                                                                   'p2','p2_dog',
                                                                                   'p3','p3_dog']]

,img_num,tweet_id,p1,p1_dog,p2,p2_dog,p3,p3_dog
6,1,666051853826850816,box_turtle,False,mud_turtle,False,terrapin,False
17,1,666104133288665088,hen,False,cock,False,partridge,False
18,1,666268910803644416,desktop_computer,False,desk,False,bookcase,False
21,1,666293911632134144,three-toed_sloth,False,otter,False,great_grey_owl,False
25,1,666362758909284353,guinea_pig,False,skunk,False,hamster,False
...,...,...,...,...,...,...,...,...
2021,1,880935762899988482,street_sign,False,umbrella,False,traffic_light,False
2022,1,881268444196462592,tusker,False,Indian_elephant,False,ibex,False
2046,1,886680336477933568,convertible,False,sports_car,False,car_wheel,False
2052,1,887517139158093824,limousine,False,tow_truck,False,shopping_cart,False


* `df_tweepy_extractions`

In [30]:
df_tweepy_extractions.sample(10)

,tweet_id,retweet_count,favorite_count,favorites_count_retweet,retweet_count_retweet,entities_name,entities_screen_name,entities_type,entities_user_id,mentions_user_id,mentions_name,mentions_screen_name,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet
2311,666293911632134144,327.0,472.0,0.0,0.0,,,,,,,,,,,
1855,675047298674663426,324.0,1056.0,0.0,0.0,,,,,,,,,,,
63,879674319642796034,10.0,300.0,0.0,0.0,,,,,[3105440746],[Kent],[Kent],,,,
424,819924195358416896,5056.0,13326.0,0.0,0.0,,,,,,,,,,,
2305,666396247373291520,80.0,160.0,0.0,0.0,,,,,,,,,,,
400,822610361945911296,3057.0,15255.0,0.0,0.0,[S],[_sheltay],[user],[458960986],,,,,,,
2033,671357843010908160,147.0,395.0,0.0,0.0,,,,,,,,,,,
1464,693109034023534592,628.0,1743.0,0.0,0.0,,,,,[267317404],[Matt MacCarrick],[Matt MacCarrick],,,,
131,866094527597207552,8032.0,0.0,34833.0,8032.0,,,,,[4196983835],[WeRateDogs®],[WeRateDogs®],,,,
265,840370681858686976,4628.0,16729.0,0.0,0.0,[Tao Lao],[lalisimo],[user],[83671200],,,,,,,


In [31]:
df_tweepy_extractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 16 columns):
tweet_id                     2333 non-null object
retweet_count                2333 non-null float64
favorite_count               2333 non-null float64
favorites_count_retweet      2333 non-null float64
retweet_count_retweet        2333 non-null float64
entities_name                2333 non-null object
entities_screen_name         2333 non-null object
entities_type                2333 non-null object
entities_user_id             2333 non-null object
mentions_user_id             2333 non-null object
mentions_name                2333 non-null object
mentions_screen_name         2333 non-null object
quoted_status_id             2333 non-null object
quoted_user_id               2333 non-null object
quoted_status_id_rwetweet    2333 non-null object
quoted_user_id_retweet       2333 non-null object
dtypes: float64(4), object(12)
memory usage: 291.8+ KB


In [32]:
df_tweepy_extractions.query('quoted_user_id != ""') 

,tweet_id,retweet_count,favorite_count,favorites_count_retweet,retweet_count_retweet,entities_name,entities_screen_name,entities_type,entities_user_id,mentions_user_id,mentions_name,mentions_screen_name,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet
41,884247878851493888,18980.0,69672.0,0.0,0.0,,,,,,,,883965650754039809,3097795571,,
71,878604707211726852,6678.0,28880.0,0.0,0.0,,,,,,,,878599868507402241,742143,,
82,876537666061221889,4376.0,22588.0,0.0,0.0,,,,,,,,876162994446753793,507967020,,
87,875097192612077568,5694.0,26314.0,0.0,0.0,,,,,,,,874413398133547008,21357526,,
106,871102520638267392,5209.0,20174.0,0.0,0.0,,,,,,,,871075758080503809,2745835322,,
127,866720684873056260,4588.0,19450.0,0.0,0.0,,,,,,,,866458718883467265,14173315,,
184,855818117272018944,5332.0,26359.0,0.0,0.0,,,,,,,,855656431005061120,184136149,,
232,846505985330044928,3088.0,14241.0,0.0,0.0,,,,,,,,846484798663245829,264924050,,
234,846139713627017216,10079.0,31307.0,0.0,0.0,,,,,,,,846088479142531073,2319349099,,
257,841320156043304961,5579.0,20097.0,0.0,0.0,,,,,,,,841311395547250688,28785486,,


In [33]:
df_tweepy_extractions.query('retweet_count_retweet != ""').shape

(2333, 16)

* `df_scrapped_replies`

In [34]:
df_scrapped_replies.sample(10)

,conversation,favs,full_name,image,language,references,replies,reply_id,retweets,text,timestamp,user_id,user_name
12039,867421006826221569,12.0,Rebecca *Impeach the Mother F$%@er* H.,NaN,en,"['1337271', '4196983835', '2300530382']",0.0,867422691418099718,0.0,"Walter likes to steal Skittles from my bowl, l...",2017-05-24 18:51:00,30219648,@R_Claw
47875,745433870967832576,1.0,Isspooky,NaN,en,['4196983835'],0.0,745437548684656640,0.0,it's the asexual pride dog,2016-06-22 04:05:35,275640445,@Forgotemme
72174,667182792070062081,3.0,jessie.js,NaN,en,['4196983835'],0.0,707431403588341760,0.0,50 laughs,2016-03-09 06:02:44,110298748,@jessiedotjs
71968,667549055577362432,1.0,P-NUT,NaN,en,['4196983835'],0.0,745723279667695616,0.0,Would definitely not pet,2016-06-22 23:00:59,2941100146,@BgnHunter
1354,890240255349198849,0.0,loving kindness,NaN,en,['4196983835'],0.0,890516285112668160,0.0,Notice elevated pinky.bougie af,2017-07-27 12:16:42,2719166986,@nohostages
59849,689283819090870273,1.0,Durbingo,NaN,en,['4196983835'],0.0,689662735013908480,0.0,Modern jet planes were modeled after this br...,2016-01-20 05:16:23,2855207540,@durbingo
68838,672622327801233409,2.0,em ☆彡,NaN,en,['4196983835'],0.0,672622737555374081,0.0,@makromolecule pupper,2015-12-04 04:45:32,2477778210,@emiline_m
33612,799757965289017345,2.0,Maneet/ਮਾਨੀਤ,NaN,en,"['4196983835', '1240167234']",0.0,799809879733059584,0.0,She is a most expressive doggo. Speaks well of...,2016-11-19 04:01:49,14090672,@maneet
53177,712092745624633345,1.0,jp,NaN,en,['4196983835'],0.0,712096790003060737,0.0,never seen that breed,2016-03-22 03:01:19,240356330,@hotspurjp
7292,880872448815771648,0.0,Rinki,NaN,en,"['4196983835', '72653944']",1.0,881035373459030017,0.0,Let them be natural ...imagine urself dressed ...,2017-07-01 08:22:56,863077956112965632,@Rinki18927810


In [35]:
df_scrapped_replies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 13 columns):
conversation    72909 non-null object
favs            72909 non-null float64
full_name       72456 non-null object
image           3864 non-null object
language        72909 non-null object
references      72909 non-null object
replies         72909 non-null float64
reply_id        72909 non-null int64
retweets        72909 non-null float64
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null int64
user_name       72909 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 7.2+ MB


#### Quality

##### `df_twitter_archive_enhanced` table:

- Some tweets are not available now. 
- The type of the columns `tweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` should be string.
- We are not intereste in the first par of the text (RT @XXXX:) when the tweet is a retweet. I already have this information in other columns.
- There are 23 rows with rating_denominator different to 10.
- And many rows have a numerator not very realistic.
- The name of the dogs `a`, `O`, `by`, `an`, `the`, `his`, `all`and `my` are incorrect. (We are not going to correct this by the moment. We do not need them)
- There are not many dogs classified as doggo, floofer, etc. And 14 of them have double clasification.


##### `df_image_predictions` table:

- The type of the column `tweet_id` should be string.


##### `df_tweepy_extractions` table:

- Nulls represented as void strings in `entities_name`,	`entities_screen_name`, `entities_type`, `entities_user_id`, `favorites_count_retweet`,	`entions_name`,	`mentions_screen_name`,	`mentions_user_id`,	`quoted_status_id`,	`quoted_status_id_rwetweet` and `retweet_count_retweet`.


##### `df_scrapped_replies` table:

- The type of the columns `favs`, `replies` and `retweets` should be integer instead of float.
- The type of the columns `user_id` and `reply_id` should be a string.
- `language` type should be categorical.
	




#### Tidiness

##### `df_twitter_archive_enhanced` table:

- Columns `doggo`, `floofer`, `pupper` and `puppo` should be a unique column called dog_type.
- This new column should be categorical.


##### `df_image_predictions` table:


##### `df_tweepy_extractions` table:

- The column `retweet_count_retweet` has no sense because it has the same value as `retweet_count`.
- we don't need the columns `entities_name`, `entities_screen_name`, `entities_type`, `mentions_name` and `mentions_screen_name` because we are going to use only the ids, so we can drop them by now.
- this dataframe should be integrated in df_twitter_archive_enhanced.

##### `df_scrapped_replies` table:

- we don't need the columns `user_name` and `full_name` because we ar goin to user only the id.

##### `df_twitter_archive_enhanced` and `df_scrapped_replies` 

- We want mentions, replies, retweets, quoted, etc grouped apart as a different kind of information. We can drop this columns in the original dataframes. We will do this at the end of the clean work.


## Clean

In [36]:
#make a copy of all dataframes
df_twitter_archive_enhanced_copy = df_twitter_archive_enhanced.copy()
df_image_predictions_copy = df_image_predictions.copy()
df_tweepy_extractions_copy = df_tweepy_extractions.copy()
df_scrapped_replies_copy = df_scrapped_replies.copy()

### Missing Data

#### `df_twitter_archive_enhanced`: Some tweets are not available now.

##### Define
Delete all rows whose tweet_id are not present in df_tweepy_extractions.

##### Code

In [37]:
df_twitter_archive_enhanced_copy[~df_twitter_archive_enhanced_copy.tweet_id.isin(df_tweepy_extractions_copy.tweet_id)].shape

(23, 17)

In [38]:
df_twitter_archive_enhanced_copy = df_twitter_archive_enhanced_copy[df_twitter_archive_enhanced_copy.tweet_id.isin(df_tweepy_extractions_copy.tweet_id)]

##### Test

In [39]:
df_twitter_archive_enhanced_copy[~df_twitter_archive_enhanced_copy.tweet_id.isin(df_tweepy_extractions_copy.tweet_id)].shape

(0, 17)

#### `df_twitter_archive_enhanced`: There are not many dogs classified as doggo, floofer, etc. And 14 of them have double clasification.

##### Define
-Try to extract this information from the comments in the replies. Maybe the most commented category in the text of df_scrapped_replies_copy can give us an oppinion about this value. Define a new column with this iformation.

-Check the tweets with two values and put the correct one.

##### Code

In [40]:
#new dataframe df_type_of_dog
df_type_of_dog = pd.DataFrame()
df_type_of_dog['conversation'] = df_scrapped_replies_copy.conversation.astype(str)
#count the times that the words doggo, floofer, etc, appear in each reply
df_type_of_dog['doggo'] = df_scrapped_replies_copy.text.str.count("doggo")
df_type_of_dog['floofer'] = df_scrapped_replies_copy.text.str.count("floofer")
df_type_of_dog['pupper'] = df_scrapped_replies_copy.text.str.count("pupper")
df_type_of_dog['puppo'] = df_scrapped_replies_copy.text.str.count("puppo")
#sum the number of words by conversation
df_type_of_dog = df_type_of_dog.groupby('conversation').sum()
#delete the rows with all 0's
df_type_of_dog = df_type_of_dog.query('(doggo == floofer == pupper == puppo == 0) == False')
#put the type with the maximum value in the column 'type'.
df_type_of_dog['type'] = df_type_of_dog[["doggo", "floofer", "pupper", "puppo"]].idxmax(axis=1)
#drop the rest of the columns
df_type_of_dog = df_type_of_dog.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'])
df_type_of_dog = df_type_of_dog.reset_index()


In [41]:
df_twitter_archive_enhanced_copy['tweet_id'] = df_twitter_archive_enhanced_copy['tweet_id'].astype(str)
#merge df_type_of_dog and df_twitter_archive_enhanced_copy
df_twitter_archive_enhanced_copy = df_twitter_archive_enhanced_copy.merge(df_type_of_dog, 
                                                                          left_on='tweet_id', 
                                                                          right_on='conversation', 
                                                                          how = 'left')

In [42]:
#number of dogs with two casifications.
df_twitter_archive_enhanced_copy[
    df_twitter_archive_enhanced_copy[["doggo","floofer","pupper","puppo"]].
                            isin(["doggo","floofer","pupper","puppo"]).sum(axis=1)> 1][['tweet_id',"doggo","floofer","pupper","puppo","type"]]

,tweet_id,doggo,floofer,pupper,puppo,type
183,855851453814013952,doggo,None,None,puppo,puppo
192,854010172552949760,doggo,floofer,None,None,floofer
444,817777686764523521,doggo,None,pupper,None,floofer
514,808106460588765185,doggo,None,pupper,None,doggo
548,802265048156610565,doggo,None,pupper,None,doggo
557,801115127852503040,doggo,None,pupper,None,NaN
687,785639753186217984,doggo,None,pupper,None,doggo
715,781308096455073793,doggo,None,pupper,None,NaN
759,775898661951791106,doggo,None,pupper,None,pupper
801,770093767776997377,doggo,None,pupper,None,doggo


In [43]:
#correction of the list above.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==855851453814013952, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==854010172552949760, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==817777686764523521, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==808106460588765185, ['pupper']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==802265048156610565, ['pupper']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==801115127852503040, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==785639753186217984, ['pupper']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==781308096455073793, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==775898661951791106, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==770093767776997377, ['pupper']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==759793422261743616, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==751583847268179968, ['pupper']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==741067306818797568, ['doggo']] = None
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']==733109485275860992, ['pupper']] = None

##### Test

In [44]:
df_twitter_archive_enhanced_copy.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,conversation,type
567,800018252395122689,NaN,NaN,2016-11-19 16:49:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo doin a struggle. 11/10 much det...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/800018252...,11,10,None,doggo,None,None,None,800018252395122689,doggo
147,862457590147678208,NaN,NaN,2017-05-11 00:01:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Jersey. He likes to watch movies, but ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/862457590...,13,10,Jersey,None,None,None,None,862457590147678208,pupper
347,829501995190984704,NaN,NaN,2017-02-09 01:27:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Leo. He was a skater pup. She said see...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/829501995...,12,10,Leo,None,None,None,None,829501995190984704,pupper
520,806629075125202948,NaN,NaN,2016-12-07 22:38:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Good afternoon class today we're going to lea...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/806629075...,13,10,None,None,None,None,None,NaN,NaN
1414,697255105972801536,NaN,NaN,2016-02-10 03:05:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Charlie. He likes to kiss all the big mil...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697255105...,10,10,Charlie,None,None,None,None,697255105972801536,pupper


#### `df_twitter_archive_enhanced` There are 23 rows with rating_denominator different to 10.

##### Define
Check the text of the tweets where the denominator is not 10.

Put the correct value in the row rating_denominator if it is possible.
It seems that many of them are correct. It uses to put denominatpor greater than 10 when there are various dogs. But many other are incorrect.

##### Code

In [45]:
df_twitter_archive_enhanced_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2332
Data columns (total 19 columns):
tweet_id                      2333 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2333 non-null object
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           165 non-null float64
retweeted_status_user_id      165 non-null float64
retweeted_status_timestamp    165 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2333 non-null int64
rating_denominator            2333 non-null int64
name                          2333 non-null object
doggo                         2333 non-null object
floofer                       2333 non-null object
pupper                        2333 non-null object
puppo                         2333 non-null object
conversation                  1

In [46]:
df_twitter_archive_enhanced_copy.query('rating_denominator != 10').tweet_id.count()

22

In [47]:
df_twitter_archive_enhanced_copy.query('rating_denominator != 10')[['tweet_id','text', 'rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
299,835246439529840640,@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0
328,832088576586297345,@docmisterio account started on 11/15/15,11,15
417,820690176645140481,The floofs have been released I repeat the flo...,84,70
499,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24,7
881,758467244762497024,Why does this never happen at my front door......,165,150
1046,740373189193256964,"After so many requests, this is Bretagne. She ...",9,11
1098,731156023742988288,Say hello to this unbelievably well behaved sq...,204,170
1143,722974582966214656,Happy 4/20 from the squad! 13/10 for all https...,4,20
1180,716439118184652801,This is Bluebert. He just saw that both #Final...,50,50
1206,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99...,99,90


In [48]:
#835246439529840640: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==835246439529840640, ['rating_numerator']] = 13
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==835246439529840640, ['rating_denominator']] = 10
#832088576586297345: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==832088576586297345, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==832088576586297345, ['rating_denominator']] = np.NaN
#820690176645140481: 84/70 It is correct.

#810984652412424192: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==810984652412424192, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==810984652412424192, ['rating_denominator']] = np.NaN
#775096608509886464: It is correct.

#740373189193256964: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==740373189193256964, ['rating_numerator']] = 14
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==740373189193256964, ['rating_denominator']] = 10
#731156023742988288: It is correct.

#722974582966214656: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==722974582966214656, ['rating_numerator']] = 13
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==722974582966214656, ['rating_denominator']] = 10

#716439118184652801: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==716439118184652801, ['rating_numerator']] = 11
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==716439118184652801, ['rating_denominator']] = 10

#713900603437621249: It is correct.
#710658690886586372: It is correct.
#709198395643068416: It is correct.
#704054845121142784: It is correct.
#697463031882764288: It is correct.
#686035780142297088: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==686035780142297088, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==686035780142297088, ['rating_denominator']] = np.NaN
#684225744407494656: It is correct.
#684222868335505415: It is correct.
#682962037429899265: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==682962037429899265, ['rating_numerator']] = 10
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==682962037429899265, ['rating_denominator']] = 10
#682808988178739200: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==682808988178739200, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==682808988178739200, ['rating_denominator']] = np.NaN
#677716515794329600: It is correct.
#675853064436391936: It is correct.
#666287406224695296: It is not the real rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==666287406224695296, ['rating_numerator']] = 9
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==666287406224695296, ['rating_denominator']] = 10

##### Test

In [49]:
df_twitter_archive_enhanced_copy.query('rating_denominator != 10')[['tweet_id','text', 'rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
299,835246439529840640,@jonnysun @Lin_Manuel ok jomny I know you're e...,960.0,0.0
328,832088576586297345,@docmisterio account started on 11/15/15,11.0,15.0
417,820690176645140481,The floofs have been released I repeat the flo...,84.0,70.0
499,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24.0,7.0
881,758467244762497024,Why does this never happen at my front door......,165.0,150.0
1046,740373189193256964,"After so many requests, this is Bretagne. She ...",9.0,11.0
1098,731156023742988288,Say hello to this unbelievably well behaved sq...,204.0,170.0
1143,722974582966214656,Happy 4/20 from the squad! 13/10 for all https...,4.0,20.0
1180,716439118184652801,This is Bluebert. He just saw that both #Final...,50.0,50.0
1206,713900603437621249,Happy Saturday here's 9 puppers on a bench. 99...,99.0,90.0


#### `df_twitter_archive_enhanced` And many rows have a numerator not very realistic.

##### Define
Check the rows with a denominator greater than 20 and confirm that are true.

##### Code

In [50]:
df_twitter_archive_enhanced_copy.query('rating_numerator > 20')[['tweet_id','text', 'rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
180,855862651834028034,@dhmontgomery We also gave snoop dogg a 420/10...,420.0,10.0
181,855860136149123072,@s8n You tried very hard to portray this good ...,666.0,10.0
278,838150277551247360,@markhoppus 182/10,182.0,10.0
299,835246439529840640,@jonnysun @Lin_Manuel ok jomny I know you're e...,960.0,0.0
326,832215909146226688,"RT @dog_rates: This is Logan, the Chow who liv...",75.0,10.0
417,820690176645140481,The floofs have been released I repeat the flo...,84.0,70.0
499,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24.0,7.0
677,786709082849828864,"This is Logan, the Chow who lived. He solemnly...",75.0,10.0
744,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. ...,27.0,10.0
881,758467244762497024,Why does this never happen at my front door......,165.0,150.0


In [51]:
#855862651834028034: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==855862651834028034, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==855862651834028034, ['rating_denominator']] = np.NaN
#855860136149123072: It is a joke (666)
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==855860136149123072, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==855860136149123072, ['rating_denominator']] = np.NaN
#838150277551247360: It has no rank
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==838150277551247360, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==838150277551247360, ['rating_denominator']] = np.NaN
#832215909146226688: It is not correct.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==832215909146226688, ['rating_numerator']] = 9.75
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==832215909146226688, ['rating_denominator']] = 10
#786709082849828864: It is not correct.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==786709082849828864, ['rating_numerator']] = 9.75
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==786709082849828864, ['rating_denominator']] = 10
#778027034220126208: It is not correct.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==778027034220126208, ['rating_numerator']] = 11.27
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==778027034220126208, ['rating_denominator']] = 10
#758467244762497024: It is correct.
#749981277374128128: It is correct, but a joke.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==749981277374128128, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==749981277374128128, ['rating_denominator']] = np.NaN
#731156023742988288: It is correct.
#713900603437621249: It is correct.
#710658690886586372: It is correct.
#709198395643068416: It is correct.
#704054845121142784: It is correct.
#697463031882764288: It is correct.
#684225744407494656 It is correct, but a joke.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==684225744407494656, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==684225744407494656, ['rating_denominator']] = np.NaN
#684222868335505415: It is correct.
#680494726643068929: It is not correct.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==680494726643068929, ['rating_numerator']] = 11.26
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==680494726643068929, ['rating_denominator']] = 10
#677716515794329600: It is correct.
#675853064436391936: It is correct.
#670842764863651840:It is but a joke.
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==670842764863651840, ['rating_numerator']] = np.NaN
df_twitter_archive_enhanced_copy.loc[df_twitter_archive_enhanced_copy['tweet_id']
                                     ==670842764863651840, ['rating_denominator']] = np.NaN


##### Test

In [52]:
df_twitter_archive_enhanced_copy.query('rating_numerator > 20')[['tweet_id','text', 'rating_numerator','rating_denominator']]

,tweet_id,text,rating_numerator,rating_denominator
180,855862651834028034,@dhmontgomery We also gave snoop dogg a 420/10...,420.0,10.0
181,855860136149123072,@s8n You tried very hard to portray this good ...,666.0,10.0
278,838150277551247360,@markhoppus 182/10,182.0,10.0
299,835246439529840640,@jonnysun @Lin_Manuel ok jomny I know you're e...,960.0,0.0
326,832215909146226688,"RT @dog_rates: This is Logan, the Chow who liv...",75.0,10.0
417,820690176645140481,The floofs have been released I repeat the flo...,84.0,70.0
499,810984652412424192,Meet Sam. She smiles 24/7 &amp; secretly aspir...,24.0,7.0
677,786709082849828864,"This is Logan, the Chow who lived. He solemnly...",75.0,10.0
744,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. ...,27.0,10.0
881,758467244762497024,Why does this never happen at my front door......,165.0,150.0


### Tidiness

#### `df_twitter_archive_enhanced`  Columns `doggo`, `floofer`, `pupper` and `puppo` should be a unique column called dog_type.
#### This new column should be categorical.

##### Define
Put this four columns and the new one extracted from the replies in one sigle column. Give priority to the value extracted from the original tweet.

##### Code

In [53]:
#This function returns the value not null between the colums 'doggo', 'floofer', 'pupper' and 'puppo'.
#If all this columns have a null value then return the value in 'type' that we get from the replies.
def define_type (row):
    if row['doggo'] == 'doggo' :
        return 'doggo'
    elif row['floofer'] == 'floofer':
        return 'floofer'
    elif row['pupper'] == 'pupper':
        return 'puppo'
    elif row['puppo'] == 'puppo':
        return 'puppo'
    else:
        return row['type']

In [54]:
#Define a new column according the results of the function avobe.
df_twitter_archive_enhanced_copy['dog_type'] = df_twitter_archive_enhanced_copy.apply (lambda row: define_type(row), axis=1)

In [55]:
#drop the old columns that we don't need now.
df_twitter_archive_enhanced_copy.drop(columns = ['doggo', 'floofer', 'pupper', 'puppo', 'type'], inplace = True)

In [56]:
#convert the datatype into category
df_twitter_archive_enhanced_copy['dog_type'] = df_twitter_archive_enhanced_copy['dog_type'].astype('category')

##### Test

In [57]:
df_twitter_archive_enhanced_copy.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,conversation,dog_type
744,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27.0,10.0,Sophie,778027034220126208,puppo
1641,682808988178739200,6.827884e+17,4.196984e+09,2016-01-01 06:22:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I'm aware that I could've said 20/16, but here...",NaN,NaN,NaN,NaN,20.0,16.0,None,682808988178739200,pupper
1574,686286779679375361,NaN,NaN,2016-01-10 20:41:33 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",When bae calls your name from across the room....,NaN,NaN,NaN,https://vine.co/v/iMZx6aDbExn,12.0,10.0,None,NaN,NaN
1466,692919143163629568,NaN,NaN,2016-01-29 03:56:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please only send in dogs. Don't submit other t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/692919143...,9.0,10.0,None,692919143163629568,pupper
669,788039637453406209,NaN,NaN,2016-10-17 15:31:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Did... did they pick out that license plate? 1...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/788039637...,12.0,10.0,None,NaN,NaN


#### `df_tweepy_extractions` we don't need the columns `entities_name`, `entities_screen_name`, `entities_type`, `mentions_name` and `mentions_screen_name` because we are going to use only the ids, so we can drop them by now.

##### Define
Drop the columns entities_name, entities_screen_name, entities_type, mentions_name and mentions_screen_name.

##### Code

In [58]:
df_tweepy_extractions_copy.drop(columns = ['entities_name', 
                                           'entities_screen_name', 
                                           'entities_type', 
                                           'mentions_name', 
                                           'mentions_screen_name'], 
                                inplace = True)

##### Test

In [59]:
df_tweepy_extractions_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 11 columns):
tweet_id                     2333 non-null object
retweet_count                2333 non-null float64
favorite_count               2333 non-null float64
favorites_count_retweet      2333 non-null float64
retweet_count_retweet        2333 non-null float64
entities_user_id             2333 non-null object
mentions_user_id             2333 non-null object
quoted_status_id             2333 non-null object
quoted_user_id               2333 non-null object
quoted_status_id_rwetweet    2333 non-null object
quoted_user_id_retweet       2333 non-null object
dtypes: float64(4), object(7)
memory usage: 200.6+ KB


####  `df_tweepy_extractions` The column `retweet_count_retweet` has no sense because it has the same value as `retweet_count`.

##### Define
Drop the column `retweet_count_retweet`.

##### Code

In [60]:
df_tweepy_extractions_copy = df_tweepy_extractions_copy.drop(columns = ['retweet_count_retweet'])

##### Test

In [61]:
df_tweepy_extractions_copy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 10 columns):
tweet_id                     2333 non-null object
retweet_count                2333 non-null float64
favorite_count               2333 non-null float64
favorites_count_retweet      2333 non-null float64
entities_user_id             2333 non-null object
mentions_user_id             2333 non-null object
quoted_status_id             2333 non-null object
quoted_user_id               2333 non-null object
quoted_status_id_rwetweet    2333 non-null object
quoted_user_id_retweet       2333 non-null object
dtypes: float64(3), object(7)
memory usage: 182.4+ KB


#### `df_tweepy_extractions` This dataframe should be integrated into `df_twitter_archive_enhanced.`

##### Define

Merge both dataframes using the tweet_id as the common key.

##### Code

In [63]:
#We make a merge of the two dataframes.
df_twitter_archive_enhanced_copy = df_twitter_archive_enhanced_copy.merge(df_tweepy_extractions_copy, 
                                                                          left_on='tweet_id', 
                                                                          right_on='tweet_id', 
                                                                          how = 'inner')

##### Test

In [64]:
df_twitter_archive_enhanced_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2332
Data columns (total 24 columns):
tweet_id                      2333 non-null object
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2333 non-null object
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           165 non-null float64
retweeted_status_user_id      165 non-null float64
retweeted_status_timestamp    165 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2333 non-null float64
rating_denominator            2333 non-null float64
name                          2333 non-null object
conversation                  1285 non-null object
dog_type                      1353 non-null category
retweet_count                 2333 non-null float64
favorite_count                2333 non-null float64
favorites_count_retweet

#### `df_scrapped_replies`we don't need the columns `user_name` and `full_name` because we ar goin to user only the id.

##### Define
Drop the columns user_name and full_name from this table.

##### Code

In [65]:
df_scrapped_replies_copy.drop(columns = ['user_name', 'full_name'], inplace = True)

##### Test

In [66]:
df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 11 columns):
conversation    72909 non-null object
favs            72909 non-null float64
image           3864 non-null object
language        72909 non-null object
references      72909 non-null object
replies         72909 non-null float64
reply_id        72909 non-null int64
retweets        72909 non-null float64
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null int64
dtypes: float64(3), int64(2), object(6)
memory usage: 6.1+ MB


### Quality

#### `df_twitter_archive_enhanced` The type of the columns `tweet_id`, `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` should be string.

##### Define
Change the type of these columns to `str`.

##### Code

In [69]:
df_twitter_archive_enhanced_copy.in_reply_to_status_id = df_twitter_archive_enhanced_copy.in_reply_to_status_id.map(
    lambda x: '{:.0f}'.format(x))
df_twitter_archive_enhanced_copy.in_reply_to_user_id = df_twitter_archive_enhanced_copy.in_reply_to_user_id.map(
    lambda x: '{:.0f}'.format(x))
df_twitter_archive_enhanced_copy.retweeted_status_id = df_twitter_archive_enhanced_copy.retweeted_status_id.map(
    lambda x: '{:.0f}'.format(x))
df_twitter_archive_enhanced_copy.retweeted_status_user_id = df_twitter_archive_enhanced_copy.retweeted_status_user_id.map(
    lambda x: '{:.0f}'.format(x))


##### Test

In [70]:
df_twitter_archive_enhanced_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2332
Data columns (total 24 columns):
tweet_id                      2333 non-null object
in_reply_to_status_id         2333 non-null object
in_reply_to_user_id           2333 non-null object
timestamp                     2333 non-null object
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           2333 non-null object
retweeted_status_user_id      2333 non-null object
retweeted_status_timestamp    165 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2333 non-null float64
rating_denominator            2333 non-null float64
name                          2333 non-null object
conversation                  1285 non-null object
dog_type                      1353 non-null category
retweet_count                 2333 non-null float64
favorite_count                2333 non-null float64
favorites_count_retwe

In [71]:
df_twitter_archive_enhanced_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2332
Data columns (total 24 columns):
tweet_id                      2333 non-null object
in_reply_to_status_id         2333 non-null object
in_reply_to_user_id           2333 non-null object
timestamp                     2333 non-null object
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           2333 non-null object
retweeted_status_user_id      2333 non-null object
retweeted_status_timestamp    165 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2333 non-null float64
rating_denominator            2333 non-null float64
name                          2333 non-null object
conversation                  1285 non-null object
dog_type                      1353 non-null category
retweet_count                 2333 non-null float64
favorite_count                2333 non-null float64
favorites_count_retwe

####  `df_twitter_archive_enhanced`  We are not intereste in the first par of the text (RT @XXXX:) when the tweet is a retweet. I already have this information in other columns.

##### Define

Remove any substring with the format `RT @XXXX` from the column `text`. 

##### Code

In [72]:
#search all the substrings of the type 'RT @XXXXX':
df_twitter_archive_enhanced_copy[df_twitter_archive_enhanced_copy['text'].str.contains('RT\s(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)')]

C:\Users\jesus\Anaconda3\envs\wrangling\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,dog_type,retweet_count,favorite_count,favorites_count_retweet,entities_user_id,mentions_user_id,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet
31,886054160059072513,nan,nan,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,886053734421102592,19607400,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,...,NaN,101.0,0.0,1482.0,,[19607400],886053434075471873,,886053434075471873,4196983835
35,885311592912609280,nan,nan,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,830583320585068544,4196983835,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,...,pupper,17287.0,0.0,68612.0,[437072817],[4196983835],,,,
67,879130579576475649,nan,nan,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,878057613040115712,4196983835,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,...,doggo,6358.0,0.0,40266.0,,[4196983835],,,,
72,878404777348136964,nan,nan,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,878281511006478336,4196983835,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",...,pupper,1203.0,0.0,7390.0,,[4196983835],,,,
73,878316110768087041,nan,nan,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,669000397445533696,4196983835,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,...,pupper,6237.0,0.0,20543.0,,[4196983835],,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,746521445350707200,nan,nan,2016-06-25 01:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Shaggy. He knows exactl...,667866724293877760,4196983835,2015-11-21 00:46:50 +0000,https://twitter.com/dog_rates/status/667866724...,...,doggo,1004.0,0.0,2961.0,,[4196983835],,,,
1021,743835915802583040,nan,nan,2016-06-17 16:01:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Extremely intelligent dog here....,667138269671505920,4196983835,2015-11-19 00:32:12 +0000,https://twitter.com/dog_rates/status/667138269...,...,NaN,2129.0,0.0,4490.0,,[4196983835],,,,
1220,711998809858043904,nan,nan,2016-03-21 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @twitter: @dog_rates Awesome Tweet! 12/10. ...,711998279773347840,783214,2016-03-21 19:29:52 +0000,https://twitter.com/twitter/status/71199827977...,...,pupper,128.0,0.0,929.0,,"[783214, 4196983835]",,,,
2236,667550904950915073,nan,nan,2015-11-20 03:51:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @dogratingrating: Exceptional talent. Origi...,667548695664070656,4296831739,2015-11-20 03:43:06 +0000,https://twitter.com/dogratingrating/status/667...,...,NaN,34.0,0.0,171.0,[4196983835],[4296831739],,,,


In [73]:
#remove all the substrings of the type 'RT @XXXXX':
df_twitter_archive_enhanced_copy['text'] = df_twitter_archive_enhanced_copy['text'].str.replace('RT\s(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)', "")

In [74]:
#search all the substrings of the type '@XXXXX':
df_twitter_archive_enhanced_copy[df_twitter_archive_enhanced_copy['text'].str.contains('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,dog_type,retweet_count,favorite_count,favorites_count_retweet,entities_user_id,mentions_user_id,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet
29,886267009285017600,886266357075128320,2281181600,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,nan,nan,NaN,NaN,...,NaN,4.0,115.0,0.0,,"[2281181600, 38308544]",,,,
54,881633300179243008,881607037314052096,47384430,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,nan,nan,NaN,NaN,...,NaN,7.0,120.0,0.0,,[47384430],,,,
61,880095782870896641,nan,nan,2017-06-28 16:09:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please don't send in photos without dogs in th...,nan,nan,NaN,https://twitter.com/dog_rates/status/880095782...,...,pupper,4044.0,26440.0,0.0,,[880096313915752450],,,,
63,879674319642796034,879553827334172672,3105440746,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,nan,nan,NaN,NaN,...,NaN,10.0,300.0,0.0,,[3105440746],,,,
77,877611172832227328,nan,nan,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",: @dog_rates the boyfriend and his soaking wet...,876850772322988032,512804507,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,...,puppo,74.0,0.0,1765.0,,"[512804507, 4196983835]",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,686286779679375361,nan,nan,2016-01-10 20:41:33 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",When bae calls your name from across the room....,nan,nan,NaN,https://vine.co/v/iMZx6aDbExn,...,NaN,1687.0,4069.0,0.0,,[2252694938],,,,
1585,685663452032069632,nan,nan,2016-01-09 03:24:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Brooks. He's confused by the almighty bal...,nan,nan,NaN,https://twitter.com/dog_rates/status/685663452...,...,pupper,1486.0,3310.0,0.0,,[2594027781],,,,
1861,674800520222154752,nan,nan,2015-12-10 03:59:15 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tedders. He broke his leg saving babie...,nan,nan,NaN,https://twitter.com/dog_rates/status/674800520...,...,NaN,855.0,3005.0,0.0,,[822215679726100480],,,,
1891,674330906434379776,665814696700723200,16374678,2015-12-08 20:53:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",13/10\n@ABC7,nan,nan,NaN,NaN,...,NaN,72.0,558.0,0.0,,[16374678],,,,


In [75]:
#remove all the substrings of the type '@XXXXX':
df_twitter_archive_enhanced_copy['text'] = df_twitter_archive_enhanced_copy['text'].str.replace('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)', "")

##### Test

In [76]:
#search all the substrings of the type '@XXXXX':
df_twitter_archive_enhanced_copy[df_twitter_archive_enhanced_copy['text'].str.contains('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)')]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,dog_type,retweet_count,favorite_count,favorites_count_retweet,entities_user_id,mentions_user_id,quoted_status_id,quoted_user_id,quoted_status_id_rwetweet,quoted_user_id_retweet


####  `df_image_predictions` The type of the column `tweet_id` should be string.

##### Define
Change the data type of the column.

##### Code

In [77]:
df_image_predictions_copy.tweet_id = df_image_predictions_copy.tweet_id.map(lambda x: '{:.0f}'.format(x))

##### Test

In [78]:
df_image_predictions_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


#### `df_scrapped_replies` The type of the columns `favs`, `replies` and `retweets` should be integer instead of float.

##### Define
Change the type of these columns.

##### Code

In [79]:
df_scrapped_replies_copy.favs = df_scrapped_replies_copy.favs.astype(int)
df_scrapped_replies_copy.replies = df_scrapped_replies_copy.replies.astype(int)
df_scrapped_replies_copy.retweets = df_scrapped_replies_copy.retweets.astype(int)

##### Test

In [80]:
df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 11 columns):
conversation    72909 non-null object
favs            72909 non-null int32
image           3864 non-null object
language        72909 non-null object
references      72909 non-null object
replies         72909 non-null int32
reply_id        72909 non-null int64
retweets        72909 non-null int32
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null int64
dtypes: int32(3), int64(2), object(6)
memory usage: 5.3+ MB


#### `df_scrapped_replies` The type of the columns `user_id` and `reply_id` should be a string.

##### Define
Change the type of these columns.

##### Code

In [81]:
df_scrapped_replies_copy.user_id = df_scrapped_replies_copy.user_id.map(lambda x: '{:.0f}'.format(x))
df_scrapped_replies_copy.reply_id = df_scrapped_replies_copy.reply_id.map(lambda x: '{:.0f}'.format(x))

##### Test

In [82]:
df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 11 columns):
conversation    72909 non-null object
favs            72909 non-null int32
image           3864 non-null object
language        72909 non-null object
references      72909 non-null object
replies         72909 non-null int32
reply_id        72909 non-null object
retweets        72909 non-null int32
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null object
dtypes: int32(3), object(8)
memory usage: 5.3+ MB


#### `df_scrapped_replies` `language` type should be categorical.

##### Define
Change the type of this column.

##### Code

In [83]:
df_scrapped_replies_copy.language = df_scrapped_replies_copy.language.astype('category')

##### Test

In [84]:
df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 11 columns):
conversation    72909 non-null object
favs            72909 non-null int32
image           3864 non-null object
language        72909 non-null category
references      72909 non-null object
replies         72909 non-null int32
reply_id        72909 non-null object
retweets        72909 non-null int32
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null object
dtypes: category(1), int32(3), object(7)
memory usage: 4.8+ MB


### Tidiness

#### `df_twitter_archive_enhanced` and `df_scrapped_replies` We want mentions, replies, retweets, quoted, etc grouped apart as a different kind of information.

##### Define
Group mentions, replies, retweets, quoted, etc into another table `df_interactions`.

##### Code

In [85]:
df_interactions = pd.DataFrame()


In [86]:
#replies from we_rate_dogs to other tweets in df_twitter_archive_enhanced_copy
df_aux = df_twitter_archive_enhanced_copy[['tweet_id', 'in_reply_to_user_id', 'in_reply_to_status_id']].copy()
df_aux.rename(columns = {'in_reply_to_user_id': 'user_destiny', 'in_reply_to_status_id': 'tweet_id_origin'}, inplace = True)
df_aux['user_origin'] = '4196983835'
df_aux['interaction_type'] = 'in_reply_to_user_id'
df_aux = df_aux[df_aux.user_destiny != 'nan']

#retweets from we_rate_dogs to other tweets in df_twitter_archive_enhanced_copy
df_aux1 = df_twitter_archive_enhanced_copy[['tweet_id', 'retweeted_status_user_id', 'retweeted_status_id']].copy()
df_aux1.rename(columns = {'retweeted_status_user_id': 'user_destiny', 'retweeted_status_id': 'tweet_id_origin'}, inplace = True)
df_aux1['user_origin'] = '4196983835'
df_aux1['interaction_type'] = 'retweeted_status_user_id'
df_aux1 = df_aux1[df_aux1.user_destiny != 'nan']

#quotes from we_rate_dogs to other tweets in df_twitter_archive_enhanced_copy
df_aux2 = df_twitter_archive_enhanced_copy[['tweet_id', 'quoted_user_id', 'quoted_status_id']].copy()
df_aux2.rename(columns = {'quoted_user_id': 'user_destiny', 'quoted_status_id': 'tweet_id_origin'}, inplace = True)
df_aux2['user_origin'] = '4196983835'
df_aux2['interaction_type'] = 'quoted_user_id'
df_aux2 = df_aux2[df_aux2.user_destiny != 'nan']

In [87]:
#users labeled under the pictures from we_rate_dogs in df_twitter_archive_enhanced_copy
df_aux3 = pd.DataFrame()
for index, row in df_twitter_archive_enhanced_copy[['tweet_id', 'entities_user_id']].iterrows():
    if type(row['entities_user_id']) == list:
        for x in row['entities_user_id']:
            df_aux3 = df_aux3.append({'user_destiny': x,
                                      'tweet_id': row['tweet_id']},ignore_index=True)
df_aux3['user_origin'] = '4196983835'
df_aux3['interaction_type'] = 'entities_user_id'


In [88]:
#user mentioned in the text from we_rate_dogs in df_twitter_archive_enhanced_copy
df_aux4 = pd.DataFrame()
for index, row in df_twitter_archive_enhanced_copy[['tweet_id', 'mentions_user_id']].iterrows():
    if type(row['mentions_user_id']) == list:
        for x in row['mentions_user_id']:
            df_aux4 = df_aux4.append({'user_destiny': x,
                                      'tweet_id': row['tweet_id']},ignore_index=True)
df_aux4['user_origin'] = '4196983835'
df_aux4['interaction_type'] = 'mentions_user_id'

In [90]:
#user mentioned in the text from other users in the replies in df_scrapped_replies_copy
#It can tak several minutes.
df_aux5 = pd.DataFrame()

for index, row in df_scrapped_replies_copy[['user_id', 'reply_id', 'references']].iterrows():
    references = row['references'].split(",")
    for reference in references:
        df_aux5 = df_aux5.append({'user_destiny': reference,
                                  'tweet_id': row['reply_id'],
                                  'user_origin': row['user_id']},ignore_index=True)
df_aux5['interaction_type'] = 'references_to_user'
df_aux5 = df_aux5[df_aux5.user_destiny != '']

In [91]:
#concatenate the previus auxialiary dataframes.
df_interactions = pd.concat([df_aux, df_aux1, df_aux2, df_aux3, df_aux4, df_aux5], sort=False)

In [92]:
#drop the columns at the previous dataframes.
df_twitter_archive_enhanced_copy.drop(columns = ['in_reply_to_status_id', 
                                                 'in_reply_to_user_id',
                                                 'retweeted_status_id',
                                                 'retweeted_status_user_id',
                                                 'retweeted_status_timestamp',
                                                 'entities_user_id',
                                                 'mentions_user_id',
                                                 'quoted_status_id',
                                                 'quoted_user_id',
                                                 'quoted_status_id_rwetweet',
                                                 'quoted_user_id_retweet'],inplace = True)
df_scrapped_replies_copy.drop(columns = ['references'], inplace = True)

In [93]:

df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 10 columns):
conversation    72909 non-null object
favs            72909 non-null int32
image           3864 non-null object
language        72909 non-null category
replies         72909 non-null int32
reply_id        72909 non-null object
retweets        72909 non-null int32
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null object
dtypes: category(1), int32(3), object(6)
memory usage: 4.2+ MB


##### Test

In [94]:
df_interactions.sample(5)

,tweet_id,user_destiny,tweet_id_origin,user_origin,interaction_type
74767,743404511767760896,['4196983835'],NaN,352729613,references_to_user
63655,783334827223289856,'22872122'],NaN,286299246,references_to_user
21361,867901175756017664,'706242467755073537'],NaN,4693578204,references_to_user
15345,878781405009981440,['4196983835',NaN,17502285,references_to_user
13732,879492226786754560,'250968694'],NaN,99194607,references_to_user


In [95]:
df_scrapped_replies_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72909 entries, 0 to 72908
Data columns (total 10 columns):
conversation    72909 non-null object
favs            72909 non-null int32
image           3864 non-null object
language        72909 non-null category
replies         72909 non-null int32
reply_id        72909 non-null object
retweets        72909 non-null int32
text            65534 non-null object
timestamp       72909 non-null object
user_id         72909 non-null object
dtypes: category(1), int32(3), object(6)
memory usage: 4.2+ MB


In [96]:
df_twitter_archive_enhanced_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2332
Data columns (total 13 columns):
tweet_id                   2333 non-null object
timestamp                  2333 non-null object
source                     2333 non-null object
text                       2333 non-null object
expanded_urls              2274 non-null object
rating_numerator           2333 non-null float64
rating_denominator         2333 non-null float64
name                       2333 non-null object
conversation               1285 non-null object
dog_type                   1353 non-null category
retweet_count              2333 non-null float64
favorite_count             2333 non-null float64
favorites_count_retweet    2333 non-null float64
dtypes: category(1), float64(5), object(7)
memory usage: 239.4+ KB
